# Setup & Utils

In [420]:
!pip install --quiet reportlab
!pip install --quiet matplotlib pandas


[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [446]:
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image, Table, TableStyle, PageBreak
from reportlab.lib.styles import ParagraphStyle, getSampleStyleSheet
from reportlab.lib import colors
from reportlab.lib.units import inch
from reportlab.platypus import ListFlowable, ListItem
from reportlab.lib.pagesizes import letter, landscape, A4
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.pdfbase import pdfmetrics

from matplotlib.patches import Rectangle
from matplotlib.ticker import FixedLocator
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as mticker

import pandas as pd
import numpy as np
import os
import io

np.random.seed(42)

# Config

In [471]:
OUTPUT_PATH = "./Social_Media_Sentiment_Report.pdf"

NAVY = "#346699"
GRAY = "#808080"
MINT = "#61DDAA"
LIGHT_BLUE = '#BFDBFE'
LIGHT_RED = '#FF9E9E'
DARK_RED = "#E31A1A"
BLACK  = "#000000"

# Utils

In [423]:
def kpi_card(value, value_color=NAVY, big_label=None, small_label=None):
	fig, ax = plt.subplots(figsize=(3.5,2))
	ax.axis('off')

	# Card background
	ax.add_patch(
		Rectangle(
			(0, 0), 1, 1,
			transform=ax.transAxes,
			facecolor="#F2F2F2",   # background fill
		)
	)

	# Big Label
	if big_label:
		ax.text(
			0.025, 0.9, 
			transform=ax.transAxes,
			s=big_label,
			fontsize=12,
			fontweight='bold',
			color=NAVY
		)

	# Small Label
	if small_label:
		ax.text(
			0.025, 0.75, 
			transform=ax.transAxes,
			s=small_label,
			fontsize=12,
			color=GRAY
		)
	
	# Main value
	ax.text(
		0.5, 0.35,
		transform=ax.transAxes,
		s=value,
		ha='center', 
		va='center',
		fontsize=36,
		fontweight='bold',
		color=value_color
	)

	return fig

# # Tester
# fig = kpi_card("1,245", big_label="Total Mentions", small_label="Number of posts")


In [424]:
def fig_to_img(fig):
    buffer = io.BytesIO()
    fig.savefig(buffer, format='png', dpi=300, bbox_inches='tight')
    buffer.seek(0)

    img = Image(buffer)

    plt.close(fig)

    return img

In [425]:
def resize_img(img, width=None, height=None):
    """
    Resize ReportLab Image while keeping aspect ratio.
    width, height in points (1 inch = 72 points)
    """
    if width and height:
        ratio = min(width / img.imageWidth, height / img.imageHeight)
    elif width:
        ratio = width / img.imageWidth
    elif height:
        ratio = height / img.imageHeight
    else:
        ratio = 1  # no resizing

    img.drawWidth = img.imageWidth * ratio
    img.drawHeight = img.imageHeight * ratio
    return img


# Get Data

In [426]:
# Hardcode dummy data
data = []
for _ in range(600):
	data.append([
		np.random.choice(pd.date_range("2026-01-01", periods=30)),
		np.random.choice(["twitter", "instagram", "youtube"]),
		np.random.choice(["positive", "neutral", "negative"], p=[0.55, 0.30, 0.15]),
		"Sample post content",
		"https://example.com/post",
		np.random.choice([f"trend_{i}" for i in range(1, 501)])
	])

df = pd.DataFrame(
	data,
	columns=["DATE", "SOURCE", "SENTIMENT", "POST", "LINK", "TREND"]
)

df.head()

,DATE,SOURCE,SENTIMENT,POST,LINK,TREND
0,2026-01-07,twitter,positive,Sample post content,https://example.com/post,trend_72
1,2026-01-29,twitter,positive,Sample post content,https://example.com/post,trend_467
2,2026-01-23,youtube,positive,Sample post content,https://example.com/post,trend_373
3,2026-01-04,youtube,positive,Sample post content,https://example.com/post,trend_258
4,2026-01-24,instagram,positive,Sample post content,https://example.com/post,trend_192


# Create Separate Elements

In [427]:
figs = []

## "Total Mention" Card

In [428]:
fig = kpi_card("1,245", big_label="Total Mentions", small_label="Number of posts")
figs.append(fig_to_img(fig))

## "Daily Percentage" Card

In [429]:
# TODO: color red (negative) vs green (positive)
fig = kpi_card("-38.97%", value_color=MINT, big_label="Daily Percentage", small_label="Negative Sentiment")
figs.append(fig_to_img(fig))

## "Weekly Percentage" Card

In [430]:
# TODO: color red (negative) vs green (positive)
fig = kpi_card("-66.67%", value_color=MINT, big_label="Weekly Percentage", small_label="Negative Sentiment")
figs.append(fig_to_img(fig))

## "Sentiment Shared" Donut

In [431]:
def donut_chart_sentiment(labels, portion):
	fig, ax = plt.subplots(figsize=(4, 5))
	ax.axis('off')

	colors = [LIGHT_BLUE, MINT, LIGHT_RED]

	# Title
	ax.text(
		-0.1, 1.0, 
		transform=ax.transAxes,
		s='Sentiment Shared',
		fontsize=12,
		fontweight='bold',
		color=NAVY
	)

	# Pie chart
	wedges, _, _ = plt.pie(portion, colors=colors,
			autopct='%1.2f%%', pctdistance=1,)

	# Legend
	ax.legend(wedges, labels,
		loc="lower center",
		ncol= len(labels),
		bbox_to_anchor=(0.5, -0.1),
		frameon=False
	)

	# Draw circle to make donut
	centre_circle = plt.Circle((0, 0), 0.7, fc='white')
	fig.gca().add_artist(centre_circle)

	return fig

labels = ['Neutral', 'Positive', 'Negative']
portion = [40000, 50000, 70000]

fig = donut_chart_sentiment(labels, portion)
figs.append(fig_to_img(fig))

## "Daily Sentiment Movement" Line

In [432]:
def line_chart_daily_sentiment(date, percentage_pos, percentage_neg):
	date = pd.to_datetime(date)

	fig, ax = plt.subplots(figsize=(4, 4))

	# Title
	ax.text(
		-0.2, 1.1, 
		transform=ax.transAxes,
		s='Daily Sentiment Movement',
		fontsize=12,
		fontweight='bold',
		color=NAVY
	)

	# Lines
	ax.plot(date, percentage_pos, color=MINT, label='Positive %', linewidth=2)
	ax.plot(date, percentage_neg, color=LIGHT_RED, label='Negative %', linewidth=2)

	# Legend
	leg = ax.legend(
		loc='lower center',
		bbox_to_anchor=(0.4, -0.3),
		ncol=2,
		frameon=False
	)
	for line in leg.get_lines():
		line.set_linewidth(6)

	# Axes
	ax.spines['top'].set_visible(False)
	ax.spines['right'].set_visible(False)
	ax.spines['bottom'].set_visible(False)
	ax.spines['left'].set_visible(False)

	# Ticks - Every 7D
	ticks = pd.date_range(start=date.min(), end=date.max(), freq='7D')
	ax.xaxis.set_major_locator(FixedLocator(mdates.date2num(ticks)))

	# # Tick - First, middle, last
	# tick_positions = [date[0], date[-1]]
	# n = len(date)
	# if n > 2:
	#     middle_index = n // 2
	#     tick_positions.insert(1, date[middle_index])
	# ax.xaxis.set_major_locator(FixedLocator(mdates.date2num(tick_positions)))

	ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))
	ax.yaxis.set_major_formatter(mticker.FormatStrFormatter('%.0f%%'))
	ax.tick_params(axis='both', length=0, colors=GRAY)

	plt.xlabel('Date', labelpad=10)
	plt.ylabel('%')

	return fig

date = [
    "2026-01-02", "2026-01-03", "2026-01-04",
    "2026-01-05", "2026-01-06", "2026-01-07",
    "2026-01-08", "2026-01-09", "2026-01-10",
    "2026-01-11"
]
pctg_pos = [13.0, 14.1, 13.0, 15.0, 12.0, 16.9, 15.7, 17.2, 14.8, 16.3]
pctg_neg = [18.5, 17.2, 19.3, 15.8, 14.6, 13.9, 15.1, 14.2, 16.0, 14.7]

fig = line_chart_daily_sentiment(date, pctg_pos, pctg_neg)
figs.append(fig_to_img(fig))

## "Platform Distribution" Stacked Bar

In [433]:
def stacked_bar_platform_dist(labels, negative, neutral, positive):
	totals = np.array(negative) + np.array(neutral) + np.array(positive)
	order = np.argsort(totals)[::] 

	labels   = np.array(labels)[order]
	negative = np.array(negative)[order]
	neutral  = np.array(neutral)[order]
	positive = np.array(positive)[order]

	fig, ax = plt.subplots(figsize=(4, 6))

	# Title
	ax.text(
		-0.25, 1.05, 
		transform=ax.transAxes,
		s='Platform Distribution',
		fontsize=12,
		fontweight='bold',
		color=NAVY
	)

	# Stacked bars
	y = np.arange(len(labels))
	ax.barh(y, negative, color=LIGHT_RED, label='Negative')
	ax.barh(y, neutral, color=LIGHT_BLUE, left=np.array(negative), label='Neutral')
	ax.barh(y, positive, color=MINT, left=np.array(negative) + np.array(neutral),  label='Positive')

	# Axes
	ax.spines['top'].set_visible(False)
	ax.spines['right'].set_visible(False)
	ax.spines['bottom'].set_visible(False)
	ax.spines['left'].set_visible(False)

	# Ticks
	ax.tick_params(axis='both', length=0, colors=NAVY)

	ax.set_yticks(y)
	ax.set_yticklabels(labels)

	max_total = totals.max()
	mid_total = max_total / 2
	ax.set_xlim(0, max_total)
	ax.set_xticks([0, mid_total, max_total])
	ax.xaxis.set_major_formatter(
		mticker.FuncFormatter(lambda x, _: f"{x/1000:.1f}K" if x % 1000 else f"{int(x/1000)}K")
	)

	plt.xlabel('Total', labelpad=20)

	return fig

labels = ['Instagram', 'TikTok', 'YouTube', 'Twitter', 'Facebook']
negative = [12000, 85000, 5000, 42000, 160000]
neutral  = [18000, 110000, 8000, 95000, 220000]
positive = [9000, 75000, 3000, 60000, 140000]

fig = stacked_bar_platform_dist(labels, negative, neutral, positive)
figs.append(fig_to_img(fig))

## "Trend Analysis" Stacked Bar with Line

In [434]:
def stacked_bar_trend_analysis(dates, negative, neutral, positive):
	fig, ax = plt.subplots(figsize=(6, 4))

	dates = pd.to_datetime(dates)

	# Title
	ax.text(
		-0.15, 1.1, 
		transform=ax.transAxes,
		s='Trend Analysis',
		fontsize=12,
		fontweight='bold',
		color=NAVY
	)

	# Vertical stacked bar
	ax.bar(dates, negative, color=LIGHT_RED, label='Negative')
	ax.bar(dates, neutral, color=LIGHT_BLUE, bottom=negative, label='Neutral')
	ax.bar(dates, positive, color=MINT, bottom=np.array(negative) + np.array(neutral), label='Positive')

	# Line
	ax.plot(
		dates,
		negative,
		color='darkred',
		linewidth=2,
		linestyle='dashed',
		label='Negative Trend'
	)

	# Legend
	ax.legend(
		loc='lower center',
		bbox_to_anchor=(0.5, -0.30),
		ncol=4,
		frameon=False
	)

	# Axes
	ax.spines['top'].set_visible(False)
	ax.spines['right'].set_visible(False)
	ax.spines['bottom'].set_visible(False)
	ax.spines['left'].set_visible(False)

	# Ticks
	ax.tick_params(axis='both', length=0, colors=GRAY)

	ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))

	ax.yaxis.set_major_formatter(
		mticker.FuncFormatter(lambda x, _: f"{x/1000:.1f}K" if x % 1000 else f"{int(x/1000)}K")
	)

	plt.xlabel('Date', labelpad=10)
	plt.ylabel('Total', labelpad=10)

	return fig

dates = [
	"2026-01-01",
	"2026-01-02",
	"2026-01-03",
	"2026-01-04",
	"2026-01-05"
]
negative = [12000, 85000, 5000, 42000, 160000]
neutral  = [18000, 110000, 8000, 95000, 220000]
positive = [9000, 75000, 3000, 60000, 140000]

fig = stacked_bar_trend_analysis(dates, negative, neutral, positive)
figs.append(fig_to_img(fig))


## "Insight" GenAI

In [ ]:
from matplotlib.patches import FancyBboxPatch

# TODO: replace with AI summary
insight_text = "Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum."

# TODO: create text card

# Build Report

In [ ]:
# create matplotlib layout

# export matplotlib to reportlab